In [89]:
import numpy as np
import pandas as pd
#pd.set_option('max_columns', None)
#pd.set_option('max_rows', 90)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.neighbors import KNeighborsRegressor
import scipy.stats
from sklearn.preprocessing import StandardScaler
from pycaret.regression import setup, compare_models
from sklearn.model_selection import KFold, cross_val_score

from catboost import CatBoostRegressor
from sklearn.linear_model import BayesianRidge, HuberRegressor, Ridge, OrthogonalMatchingPursuit
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

import optuna

In [90]:
titanic_train = pd.read_csv('C:\\Users\\Auditor\\Desktop\\Victor\\New folder\\Data Science\\Projects\\Titanic\\train.csv')

In [91]:
titanic_test = pd.read_csv('C:\\Users\\Auditor\\Desktop\\Victor\\New folder\\Data Science\\Projects\\Titanic\\test.csv')

In [92]:
titanic_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [93]:
titanic_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [94]:
titanic_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [95]:
target = titanic_train['Survived']
test_ids = titanic_train['PassengerId']

In [96]:

titanic_train1 = titanic_train.drop(['PassengerId', 'Ticket','Survived','Name'], axis=1)
titanic_test1 = titanic_test.drop(['PassengerId','Ticket','Name'], axis=1)

In [97]:
titanic_train1

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.0,1,0,7.2500,NaN,S
1,1,female,38.0,1,0,71.2833,C85,C
2,3,female,26.0,0,0,7.9250,NaN,S
3,1,female,35.0,1,0,53.1000,C123,S
4,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,NaN,S
887,1,female,19.0,0,0,30.0000,B42,S
888,3,female,NaN,1,2,23.4500,NaN,S
889,1,male,26.0,0,0,30.0000,C148,C


In [98]:
titanic_test1

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,34.5,0,0,7.8292,NaN,Q
1,3,female,47.0,1,0,7.0000,NaN,S
2,2,male,62.0,0,0,9.6875,NaN,Q
3,3,male,27.0,0,0,8.6625,NaN,S
4,3,female,22.0,1,1,12.2875,NaN,S
...,...,...,...,...,...,...,...,...
413,3,male,NaN,0,0,8.0500,NaN,S
414,1,female,39.0,0,0,108.9000,C105,C
415,3,male,38.5,0,0,7.2500,NaN,S
416,3,male,NaN,0,0,8.0500,NaN,S


In [99]:
data1 = pd.concat([titanic_train1, titanic_test1], axis=0).reset_index(drop=True)
data1

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.0,1,0,7.2500,NaN,S
1,1,female,38.0,1,0,71.2833,C85,C
2,3,female,26.0,0,0,7.9250,NaN,S
3,1,female,35.0,1,0,53.1000,C123,S
4,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...
1304,3,male,NaN,0,0,8.0500,NaN,S
1305,1,female,39.0,0,0,108.9000,C105,C
1306,3,male,38.5,0,0,7.2500,NaN,S
1307,3,male,NaN,0,0,8.0500,NaN,S


In [100]:
target

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [101]:
data2 = data1.copy()

In [102]:
data2

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.0,1,0,7.2500,NaN,S
1,1,female,38.0,1,0,71.2833,C85,C
2,3,female,26.0,0,0,7.9250,NaN,S
3,1,female,35.0,1,0,53.1000,C123,S
4,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...
1304,3,male,NaN,0,0,8.0500,NaN,S
1305,1,female,39.0,0,0,108.9000,C105,C
1306,3,male,38.5,0,0,7.2500,NaN,S
1307,3,male,NaN,0,0,8.0500,NaN,S


In [103]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    1309 non-null   int64  
 1   Sex       1309 non-null   object 
 2   Age       1046 non-null   float64
 3   SibSp     1309 non-null   int64  
 4   Parch     1309 non-null   int64  
 5   Fare      1308 non-null   float64
 6   Cabin     295 non-null    object 
 7   Embarked  1307 non-null   object 
dtypes: float64(2), int64(3), object(3)
memory usage: 81.9+ KB


In [104]:
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

#Changing all null numeric values to mean
class MultipleColumnImputerMean(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns):
        self.columns = columns
    
    def fit(self, X, y=None):
        self.imputer = SimpleImputer(strategy='mean')
        self.imputer.fit(X[self.columns])
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy[self.columns] = self.imputer.transform(X_copy[self.columns])
        return X_copy

In [105]:
#Changing all null string values to mode

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

class MultipleColumnImputerMode(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns):
        self.columns = columns
    
    def fit(self, X, y=None):
        self.imputer = SimpleImputer(strategy='most_frequent')
        self.imputer.fit(X[self.columns])
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy[self.columns] = self.imputer.transform(X_copy[self.columns])
        return X_copy

In [106]:
#gets dummies/ one_hot_encoder
class GetDummies(BaseEstimator, TransformerMixin):
    
    def __init__(self, column_names):
        self.column_names = column_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        columns = X[self.column_names]  # Extract the specified columns

        # Apply one-hot encoding to each specified column
        dummies_list = [pd.get_dummies(columns[col], prefix=col) for col in self.column_names]

        # Concatenate one-hot encoded columns with the original DataFrame and delete dummied column
        X_encoded = pd.concat([X] + dummies_list, axis=1).drop(self.column_names, axis=1)

        return X_encoded


In [107]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('MultipleColumnImputerMean', MultipleColumnImputerMean(columns=['Age', 'Fare'])),
    ('MultipleColumnImputerMode', MultipleColumnImputerMode(columns=['Cabin'])),
    ('getdummies', GetDummies(column_names=['Sex','Embarked']))])
    

In [108]:
data2

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.0,1,0,7.2500,NaN,S
1,1,female,38.0,1,0,71.2833,C85,C
2,3,female,26.0,0,0,7.9250,NaN,S
3,1,female,35.0,1,0,53.1000,C123,S
4,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...
1304,3,male,NaN,0,0,8.0500,NaN,S
1305,1,female,39.0,0,0,108.9000,C105,C
1306,3,male,38.5,0,0,7.2500,NaN,S
1307,3,male,NaN,0,0,8.0500,NaN,S


In [109]:
data3 = pipeline.fit_transform(data2)

In [110]:
data3

,Pclass,Age,SibSp,Parch,Fare,Cabin,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.000000,1,0,7.2500,C23 C25 C27,0,1,0,0,1
1,1,38.000000,1,0,71.2833,C85,1,0,1,0,0
2,3,26.000000,0,0,7.9250,C23 C25 C27,1,0,0,0,1
3,1,35.000000,1,0,53.1000,C123,1,0,0,0,1
4,3,35.000000,0,0,8.0500,C23 C25 C27,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1304,3,29.881138,0,0,8.0500,C23 C25 C27,0,1,0,0,1
1305,1,39.000000,0,0,108.9000,C105,1,0,1,0,0
1306,3,38.500000,0,0,7.2500,C23 C25 C27,0,1,0,0,1
1307,3,29.881138,0,0,8.0500,C23 C25 C27,0,1,0,0,1


In [111]:
data3 = data3[['Pclass', 'Age', 'SibSp','Parch', 'Fare','Sex_female','Sex_male', 'Embarked_C','Embarked_Q','Embarked_S' ]]

In [112]:
data3

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.000000,1,0,7.2500,0,1,0,0,1
1,1,38.000000,1,0,71.2833,1,0,1,0,0
2,3,26.000000,0,0,7.9250,1,0,0,0,1
3,1,35.000000,1,0,53.1000,1,0,0,0,1
4,3,35.000000,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
1304,3,29.881138,0,0,8.0500,0,1,0,0,1
1305,1,39.000000,0,0,108.9000,1,0,1,0,0
1306,3,38.500000,0,0,7.2500,0,1,0,0,1
1307,3,29.881138,0,0,8.0500,0,1,0,0,1


In [113]:
#Scaling¶

#Scaling¶

scaler = StandardScaler()
scaler.fit(data3)

data4 = pd.DataFrame(scaler.transform(data3), index=data3.index, columns=data3.columns)

In [114]:
data4

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.841916,-0.611972,0.481288,-0.445000,-0.503595,-0.743497,0.743497,-0.50977,-0.32204,0.657394
1,-1.546098,0.630431,0.481288,-0.445000,0.734503,1.344995,-1.344995,1.96167,-0.32204,-1.521159
2,0.841916,-0.301371,-0.479087,-0.445000,-0.490544,1.344995,-1.344995,-0.50977,-0.32204,0.657394
3,-1.546098,0.397481,0.481288,-0.445000,0.382925,1.344995,-1.344995,-0.50977,-0.32204,0.657394
4,0.841916,0.397481,-0.479087,-0.445000,-0.488127,-0.743497,0.743497,-0.50977,-0.32204,0.657394
...,...,...,...,...,...,...,...,...,...,...
1304,0.841916,0.000000,-0.479087,-0.445000,-0.488127,-0.743497,0.743497,-0.50977,-0.32204,0.657394
1305,-1.546098,0.708081,-0.479087,-0.445000,1.461829,1.344995,-1.344995,1.96167,-0.32204,-1.521159
1306,0.841916,0.669256,-0.479087,-0.445000,-0.503595,-0.743497,0.743497,-0.50977,-0.32204,0.657394
1307,0.841916,0.000000,-0.479087,-0.445000,-0.488127,-0.743497,0.743497,-0.50977,-0.32204,0.657394


In [115]:
#split the data

In [116]:
#Split Data¶

train_final = data4.loc[:titanic_train.index.max(), :].copy()
test_final = data4.loc[titanic_test.index.max() + 1:, :].reset_index(drop=True).copy()

In [117]:
train_final

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.841916,-0.611972,0.481288,-0.445000,-0.503595,-0.743497,0.743497,-0.50977,-0.322040,0.657394
1,-1.546098,0.630431,0.481288,-0.445000,0.734503,1.344995,-1.344995,1.96167,-0.322040,-1.521159
2,0.841916,-0.301371,-0.479087,-0.445000,-0.490544,1.344995,-1.344995,-0.50977,-0.322040,0.657394
3,-1.546098,0.397481,0.481288,-0.445000,0.382925,1.344995,-1.344995,-0.50977,-0.322040,0.657394
4,0.841916,0.397481,-0.479087,-0.445000,-0.488127,-0.743497,0.743497,-0.50977,-0.322040,0.657394
...,...,...,...,...,...,...,...,...,...,...
886,-0.352091,-0.223721,-0.479087,-0.445000,-0.392417,-0.743497,0.743497,-0.50977,-0.322040,0.657394
887,-1.546098,-0.844922,-0.479087,-0.445000,-0.063719,1.344995,-1.344995,-0.50977,-0.322040,0.657394
888,0.841916,0.000000,0.481288,1.866526,-0.190364,1.344995,-1.344995,-0.50977,-0.322040,0.657394
889,-1.546098,-0.301371,-0.479087,-0.445000,-0.063719,-0.743497,0.743497,1.96167,-0.322040,-1.521159


In [118]:
test_final

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,-0.352091,0.009230,-0.479087,-0.445000,-0.392417,-0.743497,0.743497,-0.50977,-0.322040,0.657394
1,0.841916,-1.543774,-0.479087,1.866526,-0.176830,1.344995,-1.344995,-0.50977,-0.322040,0.657394
2,0.841916,0.000000,-0.479087,-0.445000,-0.491108,-0.743497,0.743497,1.96167,-0.322040,-1.521159
3,0.841916,-0.689622,-0.479087,-0.445000,-0.494250,-0.743497,0.743497,-0.50977,3.105202,-1.521159
4,0.841916,-0.068420,-0.479087,-0.445000,-0.491510,-0.743497,0.743497,-0.50977,-0.322040,0.657394
...,...,...,...,...,...,...,...,...,...,...
886,0.841916,0.000000,-0.479087,-0.445000,-0.488127,-0.743497,0.743497,-0.50977,-0.322040,0.657394
887,-1.546098,0.708081,-0.479087,-0.445000,1.461829,1.344995,-1.344995,1.96167,-0.322040,-1.521159
888,0.841916,0.669256,-0.479087,-0.445000,-0.503595,-0.743497,0.743497,-0.50977,-0.322040,0.657394
889,0.841916,0.000000,-0.479087,-0.445000,-0.488127,-0.743497,0.743497,-0.50977,-0.322040,0.657394


In [128]:
target

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [119]:
from catboost import CatBoostClassifier
from sklearn.linear_model import BayesianRidge, Ridge
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [120]:
catboost_params = {
    'iterations': 6000,
    'learning_rate': 0.005,
    'depth': 4,
    'l2_leaf_reg': 1,
    'eval_metric': 'Logloss',  # Use Logloss for binary classification
    'early_stopping_rounds': 200,
    'random_seed': 42
}

# Update BayesianRidge parameters for classification
br_params = {
    'n_iter': 304,
    'tol': 0.16864712769300896,
    'alpha_1': 5.589616542154059e-07,
    'alpha_2': 9.799343618469923,
    'lambda_1': 1.7735725582463822,
    'lambda_2': 3.616928181181732e-06
}

# Update LightGBM parameters for classification
lightgbm_params = {
    'num_leaves': 39,
    'max_depth': 2,
    'learning_rate': 0.13705339989856127,
    'n_estimators': 273
}

# Update Ridge parameters for classification (Note: Ridge is not typically used for classification)
ridge_params = {
    'alpha': 631.1412445239156
}

# Models for binary classification
binary_models = {
    "catboost": CatBoostClassifier(**catboost_params, verbose=0),
    "br": BayesianRidge(**br_params),
    "lightgbm": LGBMClassifier(**lightgbm_params),
    "ridge": Ridge(**ridge_params),
    # Add other classifiers suitable for binary classification
    "logistic_regression": LogisticRegression(),
    "random_forest": RandomForestClassifier()
}

# Train your models using the binary target labels y_binary
for model_name, model in binary_models.items():
    model.fit(train_final, target)

[LightGBM] [Info] Number of positive: 342, number of negative: 549
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000170 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 234
[LightGBM] [Info] Number of data points in the train set: 891, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383838 -> initscore=-0.473288
[LightGBM] [Info] Start training from score -0.473288
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

catboost_params = {
    'iterations': 6000,
    'learning_rate': 0.005,
    'depth': 4,
    'l2_leaf_reg': 1,
    'eval_metric':'RMSE',
    'early_stopping_rounds': 200,
    'random_seed': 42
}

br_params = {
    'n_iter': 304,
    'tol': 0.16864712769300896,
    'alpha_1': 5.589616542154059e-07,
    'alpha_2': 9.799343618469923,
    'lambda_1': 1.7735725582463822,
    'lambda_2': 3.616928181181732e-06
}

lightgbm_params = {
    'num_leaves': 39,
    'max_depth': 2,
    'learning_rate': 0.13705339989856127,
    'n_estimators': 273
}

ridge_params = {
    'alpha': 631.1412445239156
}

models = {
    "catboost": CatBoostRegressor(**catboost_params, verbose=0),
    "br": BayesianRidge(**br_params),
    "lightgbm": LGBMRegressor(**lightgbm_params),
    "ridge": Ridge(**ridge_params),
    "omp": OrthogonalMatchingPursuit()
}

In [121]:
for model_name, model in binary_models.items():
    model.fit(train_final, target)
    print(name + " trained.")

omp trained.
omp trained.
[LightGBM] [Info] Number of positive: 342, number of negative: 549
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000585 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 234
[LightGBM] [Info] Number of data points in the train set: 891, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383838 -> initscore=-0.473288
[LightGBM] [Info] Start training from score -0.473288
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further split

In [122]:
#Evaluate the results

results = {}

kf = KFold(n_splits=10)

for model_name, model in binary_models.items():
    result = np.exp(np.sqrt(-cross_val_score(model, train_final, target, scoring='neg_mean_squared_error', cv=kf)))
    results[name] = result

[LightGBM] [Info] Number of positive: 303, number of negative: 498
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000538 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 223
[LightGBM] [Info] Number of data points in the train set: 801, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.378277 -> initscore=-0.496867
[LightGBM] [Info] Start training from score -0.496867
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 305, number of negative: 497
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000387 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 229
[LightGBM] [Info] Number of data points in the train set: 802, number of u

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 309, number of negative: 493
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000456 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 225
[LightGBM] [Info] Number of data points in the train set: 802, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.385287 -> initscore=-0.467168
[LightGBM] [Info] Start training from score -0.467168
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [123]:
results

{'omp': array([1.62100429, 1.54813088, 1.60648002, 1.50762477, 1.39822107,
        1.44367802, 1.60648002, 1.60648002, 1.5280603 , 1.46548709])}

In [124]:
for name, result in results.items():
    print("----------\n" + name)
    print(np.mean(result))
    print(np.std(result))

----------
omp
1.5331646484173929
0.07452613111267642


In [125]:
final_predictions = (
    0.4 * np.exp(models['catboost'].predict(test_final)) +
    0.2 * np.exp(models['br'].predict(test_final)) +
    0.2 * np.exp(models['lightgbm'].predict(test_final)) +
    0.1 * np.exp(models['ridge'].predict(test_final)) +
    0.1 * np.exp(models['omp'].predict(test_final))
)

In [126]:
final_predictions

array([1.20643737, 1.70502251, 1.17758678, 1.1865434 , 1.12888173,
       1.51984364, 1.12778991, 1.08351689, 2.16555142, 2.21340176,
       1.16351867, 1.20515563, 1.65032617, 1.74389418, 2.15113399,
       1.13704909, 1.44263051, 2.51191821, 1.5265554 , 2.23062756,
       1.17909482, 1.24027352, 2.03478958, 1.1049443 , 1.11976406,
       2.21414266, 1.31894937, 2.22780766, 2.31479305, 1.72583775,
       2.32627477, 1.39402333, 1.15657271, 1.11917873, 1.39277573,
       1.64443936, 1.11610057, 1.23162127, 1.1875267 , 2.36063628,
       2.13036893, 1.16351867, 1.64261665, 1.15030904, 1.27113933,
       1.1706906 , 1.11610057, 1.01020114, 1.11011188, 1.30875554,
       1.14000227, 2.36947806, 1.08351689, 1.0444981 , 2.10983341,
       2.38459671, 1.62342009, 1.3539155 , 1.19416547, 1.07738695,
       1.13448602, 2.15938765, 1.01629714, 1.11011188, 1.08105533,
       1.94419807, 1.6291609 , 1.40859443, 2.65478627, 1.25380356,
       1.12427388, 1.5588851 , 1.11917873, 1.12595735, 1.33047

In [127]:
submission = pd.concat([test_ids, pd.Series(final_predictions, name='Survived')], axis=1)
submission

,PassengerId,Survived
0,1,1.206437
1,2,1.705023
2,3,1.177587
3,4,1.186543
4,5,1.128882
...,...,...
886,887,1.116101
887,888,2.580655
888,889,1.011807
889,890,1.116101


In [79]:
submission.to_csv('C:\\Users\\Auditor\\Desktop\\Victor\\New folder\\Data Science\\Projects\\Final_submission.csv'
, index=False, header=True)